In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt


from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay

url = "https://raw.githubusercontent.com/ValentineKornel/MMO/refs/heads/master/diabetes.csv"
data = pd.read_csv(url)
data.head()
data.dtypes

data.isnull().any()

y = data["Outcome"];
X = data.drop("Outcome", axis=1)

X.shape, y.shape


X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.3, random_state=17)

X_train.shape, X_valid.shape

y_train.shape, y_valid.shape

first_tree = DecisionTreeClassifier(random_state=17)

cross_val_score(first_tree, X_train, y_train, cv=5) # оценка модели с помощью кросс-валидации
np.mean(cross_val_score(first_tree, X_train, y_train, cv=5)) #среднее по пяти оценкам



first_tree.fit(X_train, y_train)

tree_valid_pred = first_tree.predict(X_valid)

accuracy_score(y_valid, tree_valid_pred)
cm = confusion_matrix(y_valid, tree_valid_pred)

TP = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
TN = cm[1,1]

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print(accuracy, precision, recall)


param_grid = {
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, None],  # Пробуем разные глубины дерева
    'max_features': [0.5, 0.7, 1]  # Пробуем разные варианты количества признаков
}

treeGrid = GridSearchCV(first_tree, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
treeGrid.fit(X_train, y_train)

treeGrid.best_score_, treeGrid.best_params_

best_tree = treeGrid.best_estimator_
best_score = best_tree.score(X_valid, y_valid)
print("Точность на валидационной выборке:", best_score)


plt.figure(figsize=(15, 8))
plot_tree(best_tree, feature_names=X.columns, class_names=["No Diabetes", "Diabetes"], filled=True)
plt.show()




first_knn = KNeighborsClassifier(n_neighbors=3)

knn_model = first_knn.fit(X_train, y_train)
knn_model.score(X_train, y_train)

tree_valid_pred = first_knn.predict(X_valid)

accuracy_score(y_valid, tree_valid_pred)
cm = confusion_matrix(y_valid, tree_valid_pred)

TP = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
TN = cm[1,1]

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print(accuracy, precision, recall)


knn_params = {"n_neighbors": range(1, 30, 1)}
knn_grid = GridSearchCV(first_knn, knn_params, cv=5)

knn_grid.fit(X_train, y_train);
knn_grid.best_score_, knn_grid.best_params_

knn_best_tree = knn_grid.best_estimator_



first_rf = RandomForestClassifier(n_estimators=40)

rf_model = first_rf.fit(X_train, y_train)

rf_model.score(X_train, y_train)

tree_valid_pred = first_knn.predict(X_valid)
accuracy_score(y_valid, tree_valid_pred)
cm = confusion_matrix(y_valid, tree_valid_pred)

TP = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
TN = cm[1,1]

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print(accuracy, precision, recall)


rf_params = {
    "n_estimators": [30, 40, 45, 50, 60],  # количество деревьев в лесу
    "max_depth": [5, 10, 20],  # максимальная глубина деревьев
}

rf_grid = GridSearchCV(first_rf, rf_params, cv=5)

rf_grid.fit(X_train, y_train)

rf_grid.best_score_, rf_grid.best_params_

rf_best_model = rf_grid.best_estimator_

ax = plt.gca()
rfc_disp = RocCurveDisplay.from_estimator(best_tree, X_train, y_train, ax=ax)
rfc_disp = RocCurveDisplay.from_estimator(knn_best_tree, X_train, y_train, ax=ax)
rfc_disp = RocCurveDisplay.from_estimator(rf_best_model, X_train, y_train, ax=ax)
plt.show()





KeyboardInterrupt: 